## Importing Libraries

In [1]:
import reverse_geocode
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import folium

## Reference for number of stolen vehicles - 45000 approximately

https://auto.economictimes.indiatimes.com/news/industry/over-5-vehicles-were-reported-stolen-every-hour-in-delhi-in-2018-police/67466017

## Dataset

In [2]:
number_of_data_points = 45000
dates = pd.date_range(start="2019-01-01", end="2019-12-31", periods=number_of_data_points).date
latitudes = np.random.uniform(low=12.8213, high=13.1823, size=number_of_data_points)
longitudes = np.random.uniform(low=80.1300, high=80.2941, size=number_of_data_points)

df = pd.DataFrame({
    "date": dates,
    "latitude": latitudes,
    "longitude": longitudes
})
df.head()

,date,latitude,longitude
0,2019-01-01,12.976945,80.180662
1,2019-01-01,12.885307,80.275099
2,2019-01-01,13.080653,80.280977
3,2019-01-01,13.161457,80.239857
4,2019-01-01,13.176132,80.141109


In [3]:
def latlong_address(row):
    response = reverse_geocode.get((row["latitude"], row["longitude"]))
    return pd.Series([response["city"], response["latitude"], response["longitude"]])

df[["city", "city_latitude", "city_longitude"]] = df.apply(latlong_address, axis=1)

In [4]:
df.head()

,date,latitude,longitude,city,city_latitude,city_longitude
0,2019-01-01,12.976945,80.180662,Madipakkam,12.96226,80.19864
1,2019-01-01,12.885307,80.275099,Injambakkam,12.91620,80.24880
2,2019-01-01,13.080653,80.280977,Chennai,13.08784,80.27847
3,2019-01-01,13.161457,80.239857,Chinnasekkadu,13.16089,80.25727
4,2019-01-01,13.176132,80.141109,Vellānūr,13.15804,80.10634


## Visualization

In [5]:
df_count = df.groupby(["city", "city_latitude", "city_longitude"]).size().reset_index(name="count")

m = folium.Map(location=[13.04, 80.27], tiles="cartodbpositron", zoom_start=11)

for idx, row in df_count.iterrows():
    folium.CircleMarker(location=(row["city_latitude"], row["city_longitude"]), radius=(row["count"] / df_count["count"].sum())*100,
        popup=f"{row['city']} : {row['count']}", color="red", fill=True, fill_color="black").add_to(m)

m